In [54]:
# csv 읽어오는 거 제한을 해제하는 코드
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [55]:
#주요 모듈 임포트
import pandas as pd

In [56]:
#전자기기 리뷰 읽기
review_df = pd.read_csv('reviews_Electronics.csv', engine='python')
review_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AKM1MP6P0OYPR,0132793040,"Vicki Gibson ""momo4""","[1, 1]",Corey Barker does a great job of explaining Bl...,5.0,Very thorough,1365811200,"04 13, 2013"
1,A2CX7LUOHB2NDG,0321732944,Bernie,"[0, 0]",While many beginner DVDs try to teach you ever...,5.0,Adobe Photoshop CS5 Crash Course with master P...,1341100800,"07 1, 2012"
2,A2NWSAGRHCP8N5,0439886341,bowmans2007,"[1, 1]",It never worked. My daughter worked to earn th...,1.0,absolutely horrible,1367193600,"04 29, 2013"
3,A2WNBOD3WNDNKT,0439886341,JAL,"[1, 1]",Some of the functions did not work properly. ...,3.0,Disappointing,1374451200,"07 22, 2013"
4,A1GI0U4ZRJA8WN,0439886341,Truthfull,"[4, 4]",Do not waste your money on this thing it is te...,1.0,TERRIBLE DONT WASTE YOUR MONEY,1334707200,"04 18, 2012"


In [57]:
#메타 데이터 읽기
meta_df = pd.read_csv('meta_ele.csv', engine = 'python')
meta_df.head()

,imUrl,categories,description,asin,title,related,salesRank,price,brand
0,http://ecx.images-amazon.com/images/I/31JIPhp%...,"[['Electronics', 'Computers & Accessories', 'C...",The Kelby Training DVD Mastering Blend Modes i...,0132793040,Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,http://ecx.images-amazon.com/images/I/31uogm6Y...,"[['Electronics', 'Computers & Accessories', 'C...",NaN,0321732944,Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,http://ecx.images-amazon.com/images/I/51k0qa8f...,"[['Electronics', 'Computers & Accessories', 'P...",Digital Organizer and Messenger,0439886341,Digital Organizer and Messenger,"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",{'Electronics': 144944},8.15,NaN
3,http://ecx.images-amazon.com/images/I/41HaAhbv...,"[['Electronics', 'Accessories & Supplies', 'Au...",The CLIKR-5 UR5U-8780L remote control is desig...,0511189877,CLIKR-5 Time Warner Cable Remote Control UR5U-...,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN,23.36,NaN
4,http://ecx.images-amazon.com/images/I/51FnRkJq...,"[['Electronics', 'GPS & Navigation', 'Vehicle ...","Like its award-winning predecessor, the Intell...",0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN,299.99,NaN


In [69]:
#디지털 카메라 카테고리 걸러내는 함수, 카테고리에 문제가 생기면 여기서 수정.
def cat_dslr(text):
    #해당 카테고리에 속하면 카테고리명을 수정하기 편하게 바꿈. 카테고리를 수정하고 싶으면 이곳에서 업데이트
    if text in ["[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Point & Shoot Digital Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Digital SLR Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Compact System Cameras']]",
                "[['Electronics', 'Camera & Photo', 'Digital Cameras', 'Medium Format Digital Cameras']]"]:
        return 'ggggg' #ggggg로 값을 바꾸어 수정하기 쉽게 만듦
    return text

# asin code 기준으로 데이터 병합 함수
def asin_merge(review, meta):
    meta['categories'] = meta['categories'].apply(cat_dslr) #찾고자 하는 카테고리 명을 설정한다.
    digit_asin=meta[meta['categories'] == 'ggggg'].loc[:,['asin','title','description']]
    reviewNproduct = review.merge(digit_asin, on='asin')
    return reviewNproduct

#칼럼명 수정후 원하는 칼럼(product_name, review, review_date, score, user)만 빼오는 함수, 추가로 필요한 데이터 있으면 이거 이용
def recolumns(df):
    df.rename(columns={
        'title':'product_name',
        'reviewText':'review',
        'reviewTime':'review_date',
        'overall':'score',
        'reviewerName':'user',
    }, inplace=True)
    return df.loc[:, ['product_name','review','review_date','score','user']]

In [73]:
#함수 적용
digit_camera_review_df=asin_merge(review_df, meta_df)
new_df=recolumns(digit_camera_review_df)

In [74]:
new_df.to_csv('camera_reviews.csv', encoding='utf-8', index=False)